# Modelos Lineales - Parte 1. Regresión

In [ ]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import sys

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "05_ModelosLineales"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "plots", CHAPTER_ID)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    os.makedirs(IMAGES_PATH, exist_ok=True)
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
# import warnings
# warnings.filterwarnings(action="ignore", message="^internal gelsd")

## Regresión lineal (univariada).

Vuelve a llamar Alex. Está agradecido. Pudo impresionar a su jefe gracias a lo que le mandamos de Random Forests, Neural Networks, etc. Lo nombraron *Chief Artificial Intelligence Officer*, a cargo de departamento de AI de la inmobiliaria. Pero está de vuelta en problemas (si no, creo que no llamaría). 

Resulta que hace un par de fin de semanas, el jefe de Alex estaba jugando al golf con otros dueños de inmobiliarias, y se mandó la parte con su recientemente creado departamento de Inteligencia Artificial, y les contó todo lo que habían logrado (gracias a nosotrxs) con respecto a la predicción de precios de casas en distritos de California. Pero se vio en un brete cuando uno de sus colegas le pidió alguna explicación respecto al funcionamiento de la predicción. ¿Qué variables son las más importantes? ¿Cómo hace la predicción?

El lunes siguiente, el jefe volvió a la inmobiliaria y entró precipitadamente en la oficina de Alex para pedirle que arme un equipo que pudiera darle algún sentido a las predicciones que habían logrado antes. Le dio permiso para contratar a una persona. Alex está contento porque logró meter a su novia, Brenda, como *Senior Parameter Explorer*, pero tienen que producir algún modelo que sea más fácil de entender para el jefe, sobre todo para que no se deschave que Brenda en realidad estudio diseño gráfico.

In [ ]:
# Volvamos a leer los datos de California
HOUSING_PATH = os.path.join(".", "datasets", "housing")

if 'google.colab' in sys.modules:
    
    import tarfile

    DOWNLOAD_ROOT = "https://raw.githubusercontent.com/IAI-UNSAM/ML_UNSAM/master/"
    HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

    !mkdir -p ./datasets/housing

    def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
        os.makedirs(housing_path, exist_ok=True)
        tgz_path = os.path.join(housing_path, "housing.tgz")
        #urllib.request.urlretrieve(housing_url, tgz_path)
        !wget https://raw.githubusercontent.com/IAI-UNSAM/ML_UNSAM/master/datasets/housing/housing.tgz -P {housing_path}
        housing_tgz = tarfile.open(tgz_path)
        housing_tgz.extractall(path=housing_path)
        housing_tgz.close()
    
    # Corramos la función
    fetch_housing_data()
    
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

housing = load_housing_data()

In [ ]:
housing.info()

In [ ]:
housing.describe().round(2)

### Coeficiente de Pearson

In [ ]:
corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
# Vamos a quedarnos solo con la mediana del ingreso y el valor de las casas.
x = housing.median_income
y = housing.median_house_value

Ahora nos podemos preguntar de dónde sale ese valor de 0.688. Ya lo mencionamos, pero ahora que sabemos algo más de distribuciones, podemos volver a verlo.

Si usamos la siguiente notación:

$$
\begin{array}{lll}
\mu_X &=& \mathbb{E}(X)\\
\sigma^2_X &=& \mathbb{E}\left[(X - \mathbb{E}(X))^2\right] = \mathrm{var}(X)\\
\text{cov}_{XY} &=& \mathbb{E}\left[(X - \mathbb{E}(X))(Y - \mathbb{E}(Y)\right]\;\;,
\end{array}
$$
el coeficiente de correlación se define como 

$$
\rho_{XY} = \mathbb{E}\left[\left(\frac{X - \mu_X}{\sigma_X}\right)\left(\frac{Y - \mu_Y}{\sigma_Y}\right)\right]\;\;.
$$

El valor de expectación $\mathbb{E}$ lo definimos para las distribuciones hace unas clases. Para el caso continuo:

$$
\mathbb{E}(X) = \int x f_X(x) \mathrm{d}x\;\;.
$$

Ahora bien, ¿qué pasa si no conocemos, como ahora, $f_X$? ¿Qué pasa si solo tenemos una muestra de esa distribución?

Entonces, tenemos que usar los datos para *estimar* el valor esperado, $\mathbb{E}(X)$. Para eso, formamos un *estadístico* (*statistic*, en singular, en inglés). Es decir, una función de los datos.

$$
\bar{X} = \frac{1}{N}\sum_{i=1}^N x_i\;\;.
$$

Cuando un estadístico se usa para aproximar el valor de una característica poblacional no conocido, se lo llama un *estimador*. En este caso el estadístico $\bar{X}$ es un *estimador* del valor de expectación de la distribución, y a veces vamos a usar la notación:

$$
\hat{\mu_X} = \bar{X}\;\;.
$$

También tenemos *estimadores* de la varianza y de la covarianza:

$$
\hat{\sigma}_X^2 = \frac{1}{N - 1}\sum_{i=1}^N (x_i - \bar{X})^2\;\;,
$$
y
$$
\hat{\mathrm{cov}}_{XY} = \frac{1}{N - 1}\sum_{i=1}^N (x_i - \bar{X})(y_i - \bar{Y})\;\;.
$$

Con todo esto, podemos calcular un estimador del coeficiente de correlación:

$$
\hat{\rho_{XY}} = r = \frac{\hat{\mathrm{cov}}_{XY}}{\hat{\sigma}_X \hat{\sigma}_Y}\;\;,
$$
que se conoce como el *coeficiente de correlación de Pearson*.

In [ ]:
# Calculamos los estimadores de los valores medios
Xbar = np.sum(x)/len(x)
Ybar = np.sum(y)/len(y)

# Calculamos los estimadores de los desvíos estándar (es decir, sqrt(cov))
sigmaX = np.sqrt(np.sum((x - Xbar)**2) / (len(x) - 1))
sigmaY = np.sqrt(np.sum((y - Ybar)**2) / (len(y) - 1))

covXY = np.sum((x - Xbar) * (y - Ybar)) / (len(x) - 1)

# Coeficiente de Pearson
r = covXY / (sigmaX * sigmaY)

print('El coeficiente de Pearson es: {:.3f}'.format(r))

En `numpy` tenemos funciones y métodos que hacen esto: `np.mean`, `np.std`, `np.cov`, `np.corrcoef`.

De ahora en más, usen esas implementaciones, porque en el fondo, `numpy` corre código en C y es mucho más rápido.

In [ ]:
print(np.corrcoef(x, y))

In [ ]:
corr_matrix['median_house_value'].sort_values(ascending=False)

El coeficiente de correlación toma valores $-1 < \rho < 1$, donde -1 indica una anticorrelación perfecta, y 1 indica una correlación perfecta.

El estimador, $r$, es muy sensible a puntos aberrantes (*outliers*), por lo que hay que mirar un poco los datos antes de calcular los coeficientes y ya.

In [ ]:
import scipy.stats
scipy.stats.spearmanr?

In [ ]:
plt.plot(x, y, '.', alpha=0.1)
plt.axhline(y.mean(), color='r')
plt.axvline(x.mean(), color='r')
plt.xlabel('Mediana del ingreso por distrito')
plt.ylabel('Mediana del precio de la casa')

In [ ]:
# Veamos como cambia $r$ si sacamos los distritos con precios que saturan
i = y < np.max(y)

plt.plot(x[i], y[i], '.', alpha=0.1)
plt.axhline(y[i].mean(), color='r')
plt.axvline(x[i].mean(), color='r')
plt.xlabel('Mediana del ingreso por distrito')
plt.ylabel('Mediana del precio de la casa')


r = np.corrcoef(x[i], y[i])[0, 1]
print('El coeficiente de Pearson, con datos corregidos, es: {:.8f}'.format(r))

Claro. Los puntos saturados, arriba a la derecha ayudan mucho a aumentar $r$ de forma espúrea.

In [ ]:
j = x < 10

plt.plot(x[i & j], y[i & j], '.', alpha=0.1)
plt.axhline(y[i & j].mean(), color='r')
plt.axvline(x[i & j].mean(), color='r')
plt.xlabel('Mediana del ingreso por distrito')
plt.ylabel('Mediana del precio de la casa')

r = np.corrcoef(x[i & j], y[i & j])[0, 1]
print('El coeficiente de Pearson, con datos corregidos, es: {:.8f}'.format(r))

### Modelo lineal sencillo.

El ingreso no es una variable ideal para determinar el precio de una casa, pero es lo mejor que tenemos, de manera individual. ¿Qué modelo podemos proponerles?

Brenda recordó algo de algunas clases de matemática que tuvo en la carrera y pensó que podía armar un modelo que tuviera esta pinta:

$$
y = a * x + b\;\;,
$$
es decir, una relación lineal entre ambas variables. En este tipo de modelo, la variable $x$ se conoce con el nombre de variable predictora. Este modelo tiene dos *parámetros*: $a$ y $b$, la pendiente y ordenada al origen, respectivamente.

Brenda encuentra en sus apuntes viejos una expresión para *ajustar* los parámetros:

$$
\begin{array}{lll}
\hat{a} &=& \sum_{i=0}^N (x_i - \bar{X}) (y_i - \bar{Y}) \left[\sum_{i=0}^N (x_i - \bar{X})^2\right]^{-1}\\
\hat{b} &=& \bar{Y} - \hat{a}\bar{X}
\end{array}
$$

Comparando con las ecuaciones de arriba, vemos que

$$
\hat{a} = \frac{\hat{\mathrm{cov}}_{XY}}{\hat{\mathrm{var}}(X)}
$$

In [ ]:
# Calculemos
ahat = np.cov(x[i], y[i])[0,1] / np.var(x[i])
bhat = np.mean(y[i]) - ahat * np.mean(x[i])

print('Los ajustes dan')
print('a = {:.3f}'.format(ahat))
print('b = {:.3f}'.format(bhat))

In [ ]:
plt.plot(x[i], y[i], '.', alpha=0.1)

xx = np.linspace(x[i].min(), x[i].max(), 3)
plt.plot(xx, xx * ahat + bhat, 'r-')
plt.xlabel('Mediana del ingreso por distrito')
plt.ylabel('Mediana del precio de la casa [$]')

In [ ]:
# Podemos ver los residuos, y calcular su dispersión
res = y[i] - (ahat * x[i] + bhat)

plt.plot(x[i], res, '.', alpha=0.1)
plt.axhline(0, color='0.2', ls=':')
plt.xlabel('Mediana del ingreso por distrito')
plt.ylabel('Residuos [O - C]')


print('Los residuos tienen una dispersión de $ {:.3f}'.format(res.std()))

**Pregunta**

- ¿Estamos contentos con el ajuste? ¿Por qué?

### ¿De dónde viene la fórmula de regresión lineal? (o ¿qué condiciones se tienen que cumplir para que la cosa funcione?)

Formalizando un poco más la cosa, podemos recordar que habíamos dicho que un modelo podía escribirse:

$$
y_i = m_i + \epsilon_i\;\;,
$$
donde $\epsilon_i$ es el término de error.

De ahora en más, vamos a llamar:
* $t_i$ a los valores de $Y$ que queremos reproducir,
* $y(x, \omega)$ al modelo, que dependerá de una vector de parámetros $\boldsymbol{\omega}$.

En este caso, estamos tomando, $y(x, \boldsymbol{\omega}) = \omega_0 + \omega_1 * x$.

Supongamos que los errores $\epsilon$ tienen media cero, $\mathbb{E}(\epsilon)=0$, y que están distribuidos de acuerdo a una normal (Gaussiana), y que tienen la misma varianza $\beta^{-1}$ **para todos los puntos** (ya veremos por qué usamos la inversa):

$$
\epsilon_i \sim \mathcal{N}(0, \beta^{-1})\;\; \text{para } i = \left\{1, \ldots, N\right\}
$$

En general, podemos pensar que el objetivo es encontrar el $y$ que mejor reproduce los $t_i$. 

Pero en términos bayesianos, podemos decir que buscamos conocer la distribución de probabilidad de las variables $t_i$, condicionada al valor correspondiente $x_i$. Bajo las hipótesis de arriba:

$$
p(t | x, \boldsymbol{\omega}, \beta) = \mathcal{N}(t | y(x, \boldsymbol{\omega}), \beta^{-1})\;\;.
$$

Noten que esta expresión corresponde a la verosimilitud del dato $t_i$. Una criterio para encontrar los parámetros del vector $\omega$ es maximizar la verosimilitud.

Si suponemos además que los datos son independientes, tenemos:

$$
p(\mathbf{t} | \mathbf{x}, \boldsymbol{\omega}, \beta) = \prod_{i=1}^N \mathcal{N}(t_i | y(x_i, \boldsymbol{\omega}), \beta^{-1})\;\;,
$$
que es la verosimilitud de nuestro problema.

En general, es más conveniente usar el logaritmo de la verosimilitud. Como el logaritmo es una función creciente, el máximo de la verosimilitud coincide con el máximo de su logaritmo.

En este caso, obtenemos:

$$
\ln p(\mathbf{t} | \mathbf{x}, \boldsymbol{\omega}, \beta) = -\frac{\beta}{2} \sum_{i=1}^{N} \left\{\left(y(x_i, \boldsymbol{\omega}) - t_i\right)^2\right\} + \frac{N}{2}\ln\beta - \frac{N}{2}\ln 2\pi\;\;.
$$

Como los últimos dos términos son constantes con respecto a los parámetros, podemos obviarlos a la hora de maximizar la verosimilitud. La tarea es, entonces, equivalente a minimizar:

$$
E(\boldsymbol{\omega}) = \frac{1}{2} \sum_{i=1}^{N} \left\{y(x_i, \boldsymbol{\omega}) - t_i\right\}^2\;\;,
$$
que se conoce como *error cuadrático medio*.

Muchas veces, es más simpático tener una función de error que tenga las mismas unidades que los datos. Por eso, definimos la *raíz del error cuadrático medio* (*root-mean-square error*, o *RMS*):

$$
E_{RMS} = \sqrt{2 * E(\boldsymbol{\omega})/N}
$$

***

**Ejercicio**

* Mostrar usando la expresión del error cuadrático medio, y el modelo lineal, que los valores de los parámetros $a$ y $b$ que maximizan la verosimilitud son los que aparecen arriba.

***

**<font color='red'>Nuevo</font>**. Usando estas expresiones, pero derivando con respecto a $\beta$, podemos mostrar que:
$$
\frac{1}{\beta_{ML}} = \frac{1}{N}\sum_{i=1}^N\left\{y(x_i, \boldsymbol{\omega}_{ML}) - t_i\right\}^2
$$

¡Ojo! Esta es la estimación de la varianza de los errores, no de la de los residuos.

***

**Ejercicio**

* Demostrar la expresión del estimador de máxima verosimilitud de la precisión $\beta$, derivando el logaritmo de la verosimilitud con respecto a $\beta$.

***

### Verificación de las hipótesis

Volvamos al caso de California y estudiemos con más detalle lo que pasa con los residuos.

En primer lugar, podemos calcular el error cuadrático medio (y su raíz) para el ajuste que hicimos arriba?

In [ ]:
E = 0.5 * np.sum(res**2)

res0 = res.copy()
print('El error cuadrático medio es {:.3f}'.format(E))
print('El root-mean-square error es ${:.3f}'.format(np.sqrt(2 * E / len(res))))

In [ ]:
invbeta = np.sum(res0**2) / len(res0)
print(np.sqrt(invbeta))

***
**Pregunta**

* ¿Cómo se les ocurre que podemos estudiar el cumplimiento de las hipótesis?
***


#### Histogramas

In [ ]:
import scipy.stats as st

In [ ]:
# Veamos la distribución de los residuos
a = plt.hist(res, 100, density=True)

# Podemos graficar una Gaussiana encima
# Vamos a plotear la _mejor_
xx = np.linspace(res.min(), res.max(), 100)
plt.plot(xx, st.norm.pdf(xx, res.mean(), res.std()), '-')

#### Diagrama Q-Q

In [ ]:
# Diagrama qq
import scipy.stats as st
# st.probplot(res, plot=plt.gca())
st.probplot((res - res.mean())/res.std(), plot=plt.gca(), fit=False)

In [ ]:
# En un caso que sí funciona
mm = st.norm.rvs(0, 1, size=5000)
# plt.hist(mm, 20)
a = st.probplot(mm, plot=plt.gca(), fit=False)

#### Plot de violín y de caja (violin / box plot)

In [ ]:
a = np.histogram(x[i], 25)

bb = []

for k in range(len(a[0])):
    if a[0][k] < 100:
        continue
    
    cond = (x[i] >= a[1][k]) & (x[i] < a[1][k+1])
    bb.append(res[cond].to_numpy())

In [ ]:
fig = plt.figure(figsize=(12,7))
ax = fig.add_subplot(111)
m = [bbb.mean() for bbb in bb]
s = [bbb.std() for bbb in bb]
plt.errorbar(np.arange(len(bb)), m, s, fmt='or')

In [ ]:
plt.figure(figsize=(12,7))
vv = plt.boxplot(bb, whis=[5, 95], showfliers=False)

In [ ]:
plt.figure(figsize=(12,7))
vv = plt.violinplot(bb, showmeans=True, showextrema=False)

### Regresión lineal con datos sintéticos (o el fin de la era de inferencia).

Vimos arriba que los datos de California no parecen estar verificando las hipótesis de la regresión lineal.

Vale entonces preguntarse:

* ¿Qué pasa en este caso?

* ¿A qué riesgos me expongo si no se verifican las hipótesis?

Para respoder eso, hagamos primero un aparte para discutir las propiedades del estimador de máxima verosimilitud.

### Aparte: Estimador de máxima verosimilitud (ML).

Cuando obtenemos el estimador $\hat\omega_0$, y $\hat\omega_1$ a partir de la minimización de la función de error cuadrática (o la maximización del logaritmo de la verosimilitud), se dice que se trata de estimadores de máxima verosimilitud.

Los estimadores de máxima verosimilitud tienen una propiedades muy convenientes para una gran número de muestras (no está garantizado en el caso un tamaño finito de muestras):

* **Invarianza frente a transformaciones del parámetro**. Esto quiere decir que si cambiamos la parametrización del problema, por ejempo, pasamos de $y(x, \boldsymbol{\omega}) = \omega_0 + \omega_1 * x$, a $y(x, \omega_0 , \tau(\omega_1)) = \omega_0 + \tau * x$ (por ejemplo, un cambio de variables de $x$), entonces $\hat{\tau(\omega_1)} = \tau(\hat\omega_1)$.


* **Consistencia**. A medida que aumenta el número de datos, el estimador de ML converge hacia los "verdaderos" valores de los parámetros.


* **Eficiencia**. El estimador de ML alcanza la mínima varianza que puede tener un estimador no sesgado (límite de Cramér-Rao).


* **Normalidad**. Asintóticamente, el estimador de ML tiene una distribución normal.

### Aparte II: covarianza de los estimadores y predicciones.

Ahora, podemos preguntarnos por la varianza de un estimador, y cómo se calcula.

Bajo algunas condiciones, más o menos generales, podemos mostrar (ver Frodesen, Cap. 9) que el estimador de máxima verosimilitud, en el limite de muchos datos, está distribuído como una distribución multinormal, centrada en los verdaderos valores y con matriz de covarianza, $\boldsymbol{\Sigma}$, cuya inversa es:

$$
\boldsymbol{\Sigma}^{-1}_{ij} = -\left. \frac{\partial^2 \ln \mathcal{L}}{\partial \omega_i \omega_j} \right\rvert_{\boldsymbol{\omega} = \boldsymbol{\omega_{ML}}}\;\;.
$$

Ufff, no se desanimen. Lo único que dice esto es que para encontrar los elementos de la matriz de covarianza, tenemos que derivar la función de error con respecto a los parámetros y evaluar en los estimadores de máxima verosimilitud. Luego, invertir la matriz resultante.

En el caso de la regresión lineal, esto da:

$$
\boldsymbol{\Sigma} = \frac{\beta^{-1}}{\sum_{i=1}^N(x_i - \bar{x})^2}\begin{pmatrix}\frac{1}{N}\sum_{i=1}^N x_i^2 & -\bar{x}\\
-\bar{x} & 1\\\end{pmatrix}
$$

O sea:

$$
\mathrm{var}(\omega_0) = \frac{1}{N\beta}\frac{\sum_{i=1}^N x_i^2}{\sum_{i=1}^N(x_i - \bar{x})^2}
$$
$$
\mathrm{var}(\omega_1) = \frac{1}{\beta}\frac{1}{\sum_{i=1}^N(x_i - \bar{x})^2}\\
$$
$$
\mathrm{cov}(\omega_0, \omega_1) = -\frac{1}{\beta}\frac{\bar x}{\sum_{i=1}^N(x_i - \bar{x})^2}\\\\
$$

Entonces, cuando calculemos las predicciones de nuestro modelo, ahora podemos asignarle también un error. Podemos hacer propagación de errorres

$$
t' = \hat\omega_0 + \hat\omega_1 \cdot x\;\;,
$$
de lo que se deduce:

$$
\mathrm{var}(t') = \beta + \Sigma_{00} + \Sigma_{11} \cdot x^2 + 2 \Sigma_{01} \cdot x\;\;.
$$

Cuando hacemos la predicción, ahora podemos también dar un error.

In [ ]:
# Definimos dos funciones útiles
def compute_covariance(x, beta):
    s = np.sum((x - x.mean())**2)
    return np.array([[np.sum(x**2)/len(x), -x.mean()],[-x.mean(), 1]]) / s / beta

def var_estimation(x, sigma_):
    return sigma_[0, 0] + sigma_[1, 1] * x**2 + 2 * sigma_[0, 1] * x

def var_prediction(x, sigma_, beta):
    return 1/beta + sigma_[0, 0] + sigma_[1, 1] * x**2 + 2 * sigma_[0, 1] * x

In [ ]:
# Compute covariance
sigma_ = compute_covariance(x[i], 1/invbeta)

# Compute prediction variance
xx = np.linspace(x[i].min(), x[i].max(), 100)
tprime = xx * ahat + bhat
vtprime = var_prediction(xx, sigma_, 1/invbeta)
vestimation = var_estimation(xx, sigma_)

# Las usamos para las predicciones
plt.plot(x[i], y[i], '.', alpha=0.1)
plt.plot(xx, tprime, 'r-')
plt.plot(xx, tprime - np.sqrt(vtprime), 'r:')
plt.plot(xx, tprime + np.sqrt(vtprime), 'r:')

plt.plot(xx, tprime - 5*np.sqrt(vestimation), color='Brown')
plt.plot(xx, tprime + 5*np.sqrt(vestimation), color='Brown')

plt.xlabel('Mediana del ingreso por distrito')
plt.ylabel('Mediana del precio de la casa [$]')

#plt.xlim(0, 10)
#plt.ylim(0, 400000)
plt.axvline(x[i].mean(), color='0.5', alpha=0.5)
plt.axhline(y[i].mean(), color='0.5', alpha=0.5)

Vemos dos conceptos diferentes de error, con sus respectivos errores. Por un lado, el error en la determinación de la recta, con errores mucho menores. Es decir, si el modelo fuera correcto, cuán bien conoceríamos la recta. Por otro lado, el error de predicción, que también incluye un término relacionado con la varianza de la distribución de los datos.

Vamos a usar estas ideas para estudiar el efecto de las hipótesis en la determinación de los parámetros del modelo.

### Datos sintéticos.

Vamos a generar sets de datos que violen las hipótesis de la regresión lineal. A saber:

1. Errores independientes.

2. Errores distribuidos como una normal.

3. Homoscedasticidad. Todos los términos de error tienen al misma varianza, que habíamos llamado $\beta^{-1}$.

In [ ]:
# Construyamos primero el modelo verdadero.
import numpy.random as rr

rr.seed(20200831)

# Parámetros de la ground truth
b = 4
m = 5
beta = 1./1.

# Número de datos
n = 100

# Variable predictora
x_ = 2 * np.random.rand(n, 1)

# El modelo real (ground truth)
t_ = b + m * x_

# Array para plots
xx = np.linspace(x_.min(), x_.max(), 100)

In [ ]:
# Funciones útiles

def plot_data_truth(t):
    plt.plot(x_, t, 'or')
    plt.plot(xx, xx * m + b, 'k-', lw=2, alpha=0.7, label='Ground Truth')
    plt.xlabel('x')
    plt.ylabel('t')
    plt.legend(loc=0)
    return

def see_results(x_, target_, title, prediction=False):
    
    lr = LinearRegression()

    # Ajustamos
    lr.fit(x_, target_)

    # Y pedimos los coeficientes
    print('$\hat\omega0$ = ',lr.intercept_[0])
    print('$\hat\omega1$ = ',lr.coef_[0, 0])

    # Verificamos que da lo mismo
    omega1 = np.sum((x_ - x_.mean()) * (target_ - target_.mean())) / np.sum((x_ - x_.mean())**2)
    omega0 = target_.mean() - omega1 * x_.mean()
    print(omega0, omega1)
    
    # Compute residuals
    res = x_ * lr.coef_[0][0] + lr.intercept_[0] - target_
    
    # Compute oversampled estimate curve
    tml = xx * lr.coef_[0][0] + lr.intercept_[0]

    # Estimate beta
    invbeta = np.sum(res**2)/len(res)
    
    # Calculemos ahora el error
    sigma_ = compute_covariance(x_, 1.0)
    vesti = var_estimation(xx, sigma_)
    
    # Plot 
    plot_data_truth(target_)
    # add estimate curve
    plt.plot(xx, tml, '-g', label='ML Fit', lw=3, alpha=0.5)
    # add errors
    plt.plot(xx, tml - 2 * np.sqrt(vesti0), ':b')
    plt.plot(xx, tml + 2 * np.sqrt(vesti0), ':b')

    # Legend
    plt.legend(loc=0)
    # title
    plt.title(title)
    
    fig1 = plt.gcf()
    
    # Veamos los residuos
    plt.figure()
    plt.plot(x_, target_ - (x_ * lr.coef_[0][0] + lr.intercept_[0]), 'or')
    plt.axhline(0.0, color='g', lw=4, alpha=0.5)
    plt.plot(xx, 2 * np.sqrt(vesti), ':b')
    plt.plot(xx, -2 * np.sqrt(vesti), ':b')
    
    if prediction:
        vpredi = var_prediction(xx, sigma_, 1/invbeta)
        plt.plot(xx, np.sqrt(vpredi),  ls=':', color='Brown')
        plt.plot(xx, -np.sqrt(vpredi), ls=':', color='Brown', label='predition error (1sigma)')
        plt.legend(loc=0)
        
    # plot ground truth curve
    plt.plot(xx, (xx * m + b) - tml, color='k', lw=2)
    plt.title(title)
    
    return fig1, plt.gcf()

**Caso 0**. Las hipótesis se cumplen.

In [ ]:
# Version 0

# Agregemos error normal a los datos
t0_ = t_ + np.random.randn(n, 1)
plot_data_truth(t0_)

In [ ]:
# Veamos si podemos obtener los estimadores de los parámetros para estos datos.
# Usemos sklearn
from sklearn.linear_model import LinearRegression

lr0 = LinearRegression()

# Para usar sklearn, el array tiene que se un vector columna
print(x_.shape)

# Ajustamos
lr0.fit(x_, t0_)

# Y pedimos los coeficientes
print('$\hat\omega0$ = ',lr0.intercept_[0])
print('$\hat\omega1$ = ',lr0.coef_[0, 0])

# Verificamos que da lo mismo
omega1 = np.sum((x_ - x_.mean()) * (t0_ - t0_.mean())) / np.sum((x_ - x_.mean())**2)
omega0 = t0_.mean() - omega1 * x_.mean()
print(omega0, omega1)

plot_data_truth(t0_)
tml = xx * omega1 + omega0
plt.plot(xx, tml, '-g', label='ML Fit', lw=3, alpha=0.5)
plt.legend(loc=0)

# Calculemos ahora el error
sigma_ = compute_covariance(x_, 1.0)
vesti0 = var_estimation(xx, sigma_)
plt.plot(xx, tml - 2 * np.sqrt(vesti0), ':b')
plt.plot(xx, tml + 2 * np.sqrt(vesti0), ':b')
plt.title('Caso 0: hipótesis ok')

# Veamos los residuos
plt.figure()
plt.plot(x_, t0_ - (x_ * omega1 + omega0), 'or')
plt.axhline(0.0, color='g', lw=4, alpha=0.5)
plt.plot(xx, 2 * np.sqrt(vesti0), ':b')
plt.plot(xx, -2 * np.sqrt(vesti0), ':b')
plt.plot(xx, (xx * m + b) - tml, color='k', lw=2)
plt.title('Caso 0: hipótesis ok')

Podemos ver que los parámetros están cerca, y que el resultado general es bueno.
Veamos que pasa cuando se empiezan a romper las hipótesis.

**Caso 1** Heterocedasticidad

In [ ]:
# Version 1
np.random.seed(20200831)
# Agregemos error normal a los datos, con varianza no fija
t1_ = t_ + np.random.randn(n, 1) * (1.0 * x_  + 0.4)
plot_data_truth(t1_)

In [ ]:
see_results(x_, t1_, 'Caso 1: heterocedasticidad', prediction=False)

**Caso 2** Errores no normales

In [ ]:
# Version 2
np.random.seed(20200830)

# Agregemos error sacado de una Student t con dos grados de libertad
t2_ = t_ + np.random.standard_t(3, size=[len(t_), 1]) / 3 # Divido por 3 para tener la misma varianza que antes/
plot_data_truth(t2_)

In [ ]:
# Comparemos lo que hacemos con lo que se supone que hacemos
xx_ = np.linspace(-10, 10, 1000)
plt.plot(xx_, st.norm.pdf(xx_), label='Normal')
plt.plot(xx_, st.t.pdf(xx_, 3), label='Student t (df = 3)')
plt.legend(loc=0)

In [ ]:
see_results(x_, t2_, 'Caso 2: non-Gaussian errors', prediction=False)

**Caso 3** Errores no independientes.

In [ ]:
# Version 3
np.random.seed(20200831)

# Agregemos error sacado de una multinormal con covarianza
# Uso un kernel cuadrático
dx = x_.flatten()[:, None] - x_.flatten()[None, :]
tau = 0.04
covmat = 0.3 * np.exp(-0.5* dx**2 / tau)
print(covmat.shape)
error = np.random.multivariate_normal(mean=x_.flatten()*0.0, cov=covmat, size=(1,)).T

t3_ = t_ + error + np.random.randn(n, 1) * 0.84
# DE nuevo, las constantes están fijadas para que la varianza sea 1
plot_data_truth(t3_)

In [ ]:
see_results(x_, t3_, 'Caso 3: non-independent errors', prediction=False)

**Conclusión**

Cuando las hipótesis no se cumplen, nos ponemos a jugar una lotería. Es decir, podemos llegar a pegarle a la solución real como no. Desaparecen las garantías y las buenas propiedads del estimador de máxima verosimilitud.

¿Cómo se resuelve esto? 

Hay técnicas, como la *Regresión Lineal Generalizada*, que permiten incorporar errores no normales. Pero esto es tema para la Licenciatura en Ciencia de Datos de UNSAM, no para el curso de Aprendizaje Automático. 

Acá nos van a preocupar otros temas....

### La era de las predicciones.

Hace ya varios años que se viene diciendo que la [teoría está muerta](https://www.wired.com/2008/06/pb-theory/), que entender sistemas complejos es un ejercicio inútil. Que lo mejor que podemos hacer es predcir el comportamiento de estos sistemas.

En ese caso, no importa tanto qué sabemos realmente del sistema, sino si podemos representar su comportamiento de manera. Veamos qué podemos decir de la predicción de los datos en los casos anteriores.

In [ ]:
see_results(x_, t0_, 'Caso 0: hipótesis ok', prediction=True)

In [ ]:
see_results(x_, t1_, 'Caso 1: heteroscedasticidad', prediction=True)

In [ ]:
see_results(x_, t2_, 'Caso 2: non-Gaussian errors', prediction=True)

In [ ]:
see_results(x_, t3_, 'Caso 3: non-independent errors', prediction=True)

Entonces, vemos que incluso si las hipótesis son incorrectas, el error en la predicción está dominado por los datos, y no por el modelo. Por lo tanto, podemos concentrarnos en predecir datos, incluso si el modelo no satisface muchas de las hipótesis.

**Nota**: yo no creo que el método científico esté obsoleto. Si quieren argumentos, acá hay algunos: (https://arxiv.org/pdf/2007.04095.pdf)

***

## Regresión lineal multivariada

La primera idea que se nos puede ocurrir es usar otras columnas del conjunto de datos de casas de California.

Recordemos un poco, y reconstruyamos los features que andaban mejor

In [ ]:
corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
housing['rooms_per_household'] = housing.total_rooms/housing.households
housing['bedrooms_per_household'] = housing.total_bedrooms/housing.households
housing['bedrooms_per_rooms'] = housing.total_bedrooms/housing.total_rooms

In [ ]:
corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
# Veamos las mejores columnas en acción
fig = plt.figure(figsize=(20,20))
j = i & ~housing.bedrooms_per_rooms.isnull()
A = pd.plotting.scatter_matrix(housing.loc[j, ['median_income', 'bedrooms_per_rooms', 'median_house_value']], 
                               diagonal='hist', hist_kwds={'bins': 50, 'histtype':'step'}, alpha=0.2,
                              figsize=(12,6))

In [ ]:
# Hagamos un predictor con estos
x1 = housing.median_income.to_numpy()[j]
x2 = housing.bedrooms_per_rooms.to_numpy()[j]
t = housing.median_house_value.to_numpy()[j]

In [ ]:
# Creo la matriz de diseños
phi = np.vstack([x1, x2]).T

# Instancio el regresor y ajusto
lr = LinearRegression()
lr.fit(phi, t)

In [ ]:
# Ok, ¿cómo nos fue con respecto al anterior?
res1 = t - lr.predict(phi)

print('Los residuos tienen una dipersión de $ {:.2f}'.format(np.sqrt(np.sum(res1**2) / len(res1))))
print('Los residuos con una sola variable preditiva eran $ {:.2f}'.format(np.sqrt(np.sum(res0**2) / len(res0))))

Uno puede preguntarse si valió la pena incluir la segunda variable. Esto también es tema para un curso de data science...

¡Qué continue la predicción!

### Regresión polinomial

Una cosa interesante que podemos hacer es usar no solo las variables lineales, sino también potencias de ellas (ahora veremos en las diapos cómo es que esto sigue funcionando).

Para eso, podemos usar unos truquitos de `skelearn`

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

pf = PolynomialFeatures(degree=2)
phi2 = pf.fit_transform(phi)

# Antes
print(phi.shape)
# Despues
print(phi2.shape)

`PolynomialFeatures` no solo calcula las potencias de grado dos de las variables, sino también sus cruces. Por eso, al final pasamos de dos variables a seis: $\{1, x_1, x_2, x_1 x_2, x_1^2, x_2^2\}$

In [ ]:
# Agarramos otro predictor. Miren que ahora no le pido ajustar el intercept.
lrpoly = LinearRegression(fit_intercept=False)
lrpoly.fit(phi2, t)

# Ok, ¿cómo nos fue con respecto al anterior?
res2 = t - lrpoly.predict(phi2)

print('Los residuos tienen una dipersión de $ {:.2f}'.format(np.sqrt(np.sum(res2**2) / len(res2))))
print('Los residuos con una dos variables preditivas eran $ {:.2f}'.format(np.sqrt(np.sum(res1**2) / len(res1))))

In [ ]:
# Si quieren normalizar

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pp = Pipeline([('scaler', StandardScaler()),
               ('poly', PolynomialFeatures(degree=2)),
               ('lr', LinearRegression(fit_intercept=False))
              ])

pp.fit(phi, t)

# Ok, ¿cómo nos fue con respecto al anterior?
res3 = t - pp.predict(phi)

print('Los residuos tienen una dipersión de $ {:.2f}'.format(np.sqrt(np.sum(res3**2) / len(res3))))
print('Los residuos con una dos variables preditivas eran $ {:.2f}'.format(np.sqrt(np.sum(res1**2) / len(res1))))

In [ ]:
a = plt.hist(res3, 100, density=True, histtype='step', label='2 variables')
b = plt.hist(res0, 100, density=True, histtype='step', label='1 variable')

# Podemos graficar una Gaussiana encima
# Vamos a plotear la _mejor_
xx = np.linspace(res2.min(), res2.max(), 100)
plt.plot(xx, st.norm.pdf(xx, res2.mean(), res2.std()), '-')

Bueno, el escaleo no sirve de mucho.

Pero en general la cosa sigue mejorando! Vamos bien. 
**Volvámonos locos!**

In [ ]:
k = j & ~housing.total_bedrooms.isnull()
hh = housing.loc[k].drop(axis=1, columns=['median_house_value', 'ocean_proximity'])
t = housing.loc[k, 'median_house_value']

# Let's go crazy
lr = LinearRegression()
lr.fit(hh, t)

resN = t - lr.predict(hh)
print('Los residuos tienen una dipersión de $ {:.2f}'.format(np.sqrt(np.sum(resN**2) / len(resN))))

a = plt.hist(res3, 100, density=True, histtype='step', label='2 variables')
b = plt.hist(resN, 100, density=True, histtype='step', label='N variables')

# Podemos graficar una Gaussiana encima
# Vamos a plotear la _mejor_
xx = np.linspace(res2.min(), res2.max(), 100)
plt.plot(xx, st.norm.pdf(xx, res2.mean(), res2.std()), '-')

In [ ]:
pp.fit(hh, t)

resN = t - pp.predict(hh)
print('Los residuos tienen una dipersión de $ {:.2f}'.format(np.sqrt(np.sum(resN**2) / len(resN))))

a = plt.hist(res3, 100, density=True, histtype='step', label='2 variables')
b = plt.hist(resN, 100, density=True, histtype='step', label='N variables')

# Podemos graficar una Gaussiana encima
# Vamos a plotear la _mejor_
xx = np.linspace(res2.min(), res2.max(), 100)
plt.plot(xx, st.norm.pdf(xx, res2.mean(), res2.std()), '-')
plt.legend(loc=0)

Vean como de a poco la cosa se va mejorando. Claro, que es porque aumentamos muchísimo el número de parámetros.

In [ ]:
# Tenemos once variables "de base"
print('Entrada: {}'.format(hh.shape))

# Al crear las features polinomiales
print('Fatures: {}'.format(pp.named_steps['poly'].fit_transform(hh).shape))

# Veamos cómo aumenta este número
degrees = [2, 3, 4, 5, 6]

nparams = []
for d in degrees:
    poly = PolynomialFeatures(degree=d)
    nparams.append(poly.fit_transform(hh).shape[1])
    
plt.plot(degrees, nparams, 'o-r', mfc='None', ms=10, mew=2)
plt.xlabel('Grado de features polinomiales')
plt.ylabel('Número de features / parámetros')

***

**Pregunta**

¿Qué creen que pasaría si llego a tener alrededor de 20'000 features?

*Aguanten para la respuesta*

### Sobreajuste

Para explorar la pregunta de arriba con más detalle, vamos a un caso simple, de muchos menos parámetros. Además, esto lo hace más ágil.

In [ ]:
np.random.seed(123456)

# Nuevo set de datos (esta vez, hago dos copias)
x_ = np.random.rand(20, 1)
t_ = np.sin(2*np.pi*x_) + np.random.randn(len(t_), 1) * 0.3

# Antes que nada, voy a separar en conjunto de entrenamiento y de testeo
x_train, x_test = x_[:10], x_[10:]
t_train, t_test = t_[:10], t_[10:]

In [ ]:
# Array para plotear
xx = np.linspace(0, 1, 100).reshape([-1, 1])

def plot_data_sine(x, t, ax=None):
    if ax is None:
        ax = plt.gca()
    ax.plot(x, t, 'ob', mfc='None', ms=10)
    ax.plot(xx, np.sin(2*np.pi * xx), 'g-', lw=2, alpha=0.7, label='Ground Truth')
    ax.set_xlabel('x')
    ax.set_ylabel('t')
    ax.legend(loc=0)
    return

plot_data_sine(x_train, t_train)

Vamos a hacer regresión de estos datos con features polinomiales de orden cada vez mayor.

Para eso, creo un pipeline de `sklearn`.

In [ ]:
from sklearn.metrics import mean_squared_error as mse

# Creo el pipeline
pp_sine = Pipeline([('poly', PolynomialFeatures()),
                     ('lr', LinearRegression(fit_intercept=False))])

# Inicializo listas
degrees = [1, 2, 3, 5, 7, 9, 11]
coeffs = []
rmse = []
preds = []
# Itero sobre los grados
for d in degrees:
    # Fijo el grado
    pp_sine.named_steps['poly'].degree = d
    
    # Fiteo (y registro los valores de los parámetros)
    pp_sine.fit(x_train, t_train)
    coeffs.append(pp_sine.named_steps['lr'].coef_)
    
    # Obtengo predicciones
    y_train = pp_sine.predict(x_train)
    preds.append(pp_sine.predict(xx))
    
    # Calculo el RMSE    
    rmse.append(np.sqrt(mse(t_train, y_train)))
    

In [ ]:
# Veamos cómo evoluciona la métrica con el número de grados de libertad
plt.figure(figsize=(10,4))
plt.plot(degrees, rmse, 'o-r', mfc='None', ms=10, mew=2)
plt.xlabel('Grado de features polinomiales')
plt.ylabel('RMSE')

for i, d in enumerate(degrees):
    print(d, rmse[i])

<p style="color:red; font-size:16pt">¡Bien!</p> 

O sea que cuanto mayor el grado del polinomio, mejor. Hasta llegar a cero.... ¿o no?

Veamos...

In [ ]:
ncols = 3
nrows = np.int(np.ceil(len(degrees)/ncols))

fig = plt.figure(figsize=(12, 12))
plt.subplots_adjust(hspace=0.4)

for i, d in enumerate(degrees):
    ax = fig.add_subplot(nrows, ncols, i+1)
    plot_data_sine(x_train, t_train, ax=ax)
    ax.plot(xx, preds[i], 'r-', label='Prediction')
    ax.set_ylim(-1.3, 1.3)
    ax.set_title('Grado {}'.format(d))


### Regularización